In [1]:
%load_ext autoreload
%autoreload 2

import os, re, json, ast
from llm_api_calls import send_message_to_gemini_async
from km_utils import get_ToCs, split_dict_by_size, get_content_from_articles_response, split_list_by_size
from coap_prompts import *
from parallel_descriptions import RateLimiter
from pprint import pprint

# rate_limiter = RateLimiter(9, 10)
rate_limiter = RateLimiter(13, 14)
# rate_limiter = RateLimiter(1, 10)

In [3]:
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /Users/nikolaytv/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikolaytv/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
ToCs = get_ToCs(base_path = 'coap_map')

all_articles = {}
ToC_articles = ToCs['ToC_articles']
for section in ToC_articles.keys():
    for chapter in ToC_articles[section].keys():
        for article in ToC_articles[section][chapter].keys():
            all_articles[ToC_articles[section][chapter][article]['article_title']] = ToC_articles[section][chapter][article]['article_content']


In [20]:
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Assuming nltk's data has been downloaded already
# If not, uncomment the following lines
# nltk.download('punkt')
# nltk.download('stopwords')

def preprocess(document):
    tokens = word_tokenize(document.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return stemmed_tokens


# Preprocess the documents stored as values in the dictionary
processed_docs = [preprocess(doc) for doc in all_articles.values()]

# Initialize BM25 with the preprocessed documents
bm25 = BM25Okapi(processed_docs)

# Querying
query = "что будет за незаконное хранение оружия"
tokenized_query = preprocess(query)

# Get scores for each document
scores = bm25.get_scores(tokenized_query)

# Get the top N documents' scores
n = 10
top_n_scores = sorted(scores, reverse=True)[:n]

# Match top scores to documents
top_n_docs = [doc for score in top_n_scores for doc, s in zip(all_articles.items(), scores) if s == score]

print("Top N documents:", top_n_docs)


Top N documents: [('Статья 20.13. Стрельба из оружия в отведенных для этого местах с нарушением установленных правил или в не отведенных для этого местах', '1. Стрельба из оружия в отведенных для этого местах с нарушением установленных правил -\nвлечет наложение административного штрафа в размере от трех тысяч до пяти тысяч рублей с конфискацией оружия и патронов к нему или без таковой.\n2. Стрельба из оружия в населенных пунктах или в других не отведенных для этого местах -\nвлечет наложение административного штрафа в размере от сорока тысяч до пятидесяти тысяч рублей с конфискацией оружия и патронов к нему либо лишение права на приобретение и хранение или хранение и ношение оружия на срок от полутора до трех лет с конфискацией оружия и патронов к нему.\n3. Действие, предусмотренное частью 2 настоящей статьи, совершенное группой лиц либо лицом, находящимся в состоянии опьянения, -\nвлечет наложение административного штрафа в размере от пятидесяти тысяч до ста тысяч рублей с конфискаци

In [21]:
top_n_docs

[('Статья 20.13. Стрельба из оружия в отведенных для этого местах с нарушением установленных правил или в не отведенных для этого местах',
  '1. Стрельба из оружия в отведенных для этого местах с нарушением установленных правил -\nвлечет наложение административного штрафа в размере от трех тысяч до пяти тысяч рублей с конфискацией оружия и патронов к нему или без таковой.\n2. Стрельба из оружия в населенных пунктах или в других не отведенных для этого местах -\nвлечет наложение административного штрафа в размере от сорока тысяч до пятидесяти тысяч рублей с конфискацией оружия и патронов к нему либо лишение права на приобретение и хранение или хранение и ношение оружия на срок от полутора до трех лет с конфискацией оружия и патронов к нему.\n3. Действие, предусмотренное частью 2 настоящей статьи, совершенное группой лиц либо лицом, находящимся в состоянии опьянения, -\nвлечет наложение административного штрафа в размере от пятидесяти тысяч до ста тысяч рублей с конфискацией оружия и пат

In [11]:
pprint(top_n_docs)

['Статья 2.1. Административное правонарушение',
 'Статья 32.12. Исполнение постановления об административном приостановлении '
 'деятельности',
 'Статья 15.24.1. Незаконные выдача либо обращение документов, удостоверяющих '
 'денежные и иные обязательства',
 'Статья 27.13. Задержание транспортного средства',
 'Статья 1.5. Презумпция невиновности',
 'Статья 24.5. Обстоятельства, исключающие производство по делу об '
 'административном правонарушении',
 'Статья 31.3. Обращение постановления по делу об административном '
 'правонарушении к исполнению',
 'Статья 19.34. Нарушение порядка деятельности некоммерческой организации, '
 'выполняющей функции иностранного агента',
 'Статья 2.6.2. Административная ответственность собственников или иных '
 'владельцев земельных участков либо других объектов недвижимости',
 'Статья 3.12. Административное приостановление деятельности']
